In [1]:
def main():
    print("hello world")
    
if __name__ == "__main__":
    main()

hello world


In [8]:
import sklearn
import numpy as np
import csv
import scipy.io
import pandas as pd
from pandas import ExcelFile
from pandas import ExcelWriter
import os
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC

In [21]:
rootdir = 'C:/Users/Zman/Desktop/Jiook/code_embarc/data/connectome_fmri'
cwd = os.getcwd()
print(cwd)

C:\Users\juke\Documents\GitHub\Zachs_Repo


In [37]:
import xlrd

fname_1 = 'all_fmri_pat.xlsx'

def get_list():
    corr_mats = np.array()
    names = []
    labels = []
    xl_workbook = xlrd.open_workbook(fname_1)
    sheet_names = xl_workbook.sheet_names()
    xl_sheet = xl_workbook.sheet_by_index(0)
    print(xl_sheet.cell_value(1, 1))
    print(xl_sheet.ncols, xl_sheet.nrows)
    for i in range(xl_sheet.nrows):
        if (xl_sheet.cell(i, 1).value == 0 or xl_sheet.cell(i, 1).value == 1):
            names.append(xl_sheet.cell_value(i, 0))
            labels.append(xl_sheet.cell_value(i, 3))
    
    return names, labels
    #inds = [(i for i in col_1 if (col_1[i]))]
    
names, labels = get_list()

1.0
1060 137


In [38]:
def binary_classifier(features, labels): # a simple classifier that when given a bunch of correlation matrices and labels, will predict label on new matrices.
    train, test, train_labels, test_labels = train_test_split(features, labels, test_size=0.3, random_state=42) # splits features w/ labels into training and testing sets
    gnb = GaussianNB()
    model = gnb.fit(train, train_labels)
    preds = model.pred(test, test_labels)
    print(accuracy_score(test_labels, preds))
    correct = []
    for n in len(preds):
        if (preds[n] == test_labels[n]):
            correct.append(1)
        else :
            correct.append(0)
    return correct, model

In [39]:
features, labels = get_list()
corr, mod = binary_classifier(features, labels)
m = binary_classifier(features, corr) # gives "sureness" of prediction
# problematic because this only has 33% of data. I want to record accuracy throughout process.

1.0
1060 137


ValueError: Expected 2D array, got 1D array instead:
array=['CU0046' 'TX0043' 'UM0073' 'CU0117' 'MG0172' 'CU0027' 'UM0118' 'TX0142'
 'MG0152' 'MG0101' 'CU0074' 'TX0067' 'MG0076' 'CU0034' 'MG0218' 'TX0095'
 'TX0112' 'CU0115' 'CU0083' 'TX0193' 'UM0047' 'CU0106' 'TX0110' 'MG0270'
 'CU0066' 'CU0104' 'CU0024' 'CU0077' 'MG0135' 'CU0036' 'UM0089' 'CU0033'
 'TX0101' 'UM0110' 'TX0189' 'TX0050' 'TX0014' 'UM0103' 'MG0243' 'TX0127'
 'TX0083' 'UM0119' 'TX0188' 'MG0182' 'TX0070' 'MG0257' 'UM0048' 'TX0068'
 'TX0136' 'MG0021' 'UM0090' 'TX0078' 'TX0090' 'UM0037' 'MG0126' 'CU0119'
 'UM0060' 'CU0021' 'TX0039' 'CU0090' 'CU0022' 'CU0103' 'UM0025' 'UM0074'
 'TX0133' 'TX0195' 'TX0187' 'CU0084' 'TX0079' 'TX0115' 'CU0067' 'UM0052'
 'TX0020' 'UM0107'].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

In [ ]:
def new_classifier(features, labels): # if sureness if lower than a threshold, then use classification of new
    threshold = 0.5
    train, test, train_labels, test_labels = train_test_split(features, labels, test_size=0.3, random_state=42)
    svc = SVC()
    model = svc.fit(train, train_labels)
    for i in len(test):
        if (m.pred(test[i], test_labels[i]) < threshold):
            preds = svc.pred(test[i], test_labels[i])
        else:
            preds = m.pred(test[i], test_labels[i])
    print(accuracy_score(test_labels, preds))

In [ ]:
print(binary_classifier(features, labels))
print(new_classifier(features, labels))